In [1]:
import itertools

import pandas as pd
import cobra.flux_analysis
import time
import numpy as np
from functools import partial
from src.mp_functions import combinations_subset, parallelize_dataframe, knockout_FBA


"""A mess of a document with different code cells.
Good to to use for any testing that involves the Human1 model as it takes some time to load in."""

start_time = time.time()
model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/brain.xml'
model = cobra.io.read_sbml_model(model_file_path)

end_time = time.time()
print('Model load time: %.6f seconds' % (end_time - start_time))

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic
Model load time: 32.311737 seconds


In [11]:
# Prepare SNP combinations; lists genes to knock out by model id.
combinations = pd.read_table('C:/Users/Sigve/Genome_Data/results/SNP_combinations_phewas_4_5_6.tsv', index_col=0)
combinations['phewas_code'] = combinations['phewas_code'].apply(str)
combinations = combinations[combinations['phewas_code'].str.contains('290.11')]
combinations['gene_ids'] = combinations['gene_ids'].apply(lambda x: x.split(';'))
print(combinations.shape[0])

genes = model.genes
genes = [gene.id for gene in genes]

# Removes combinations containing genes not in the model
combinations = combinations[combinations['gene_ids'].apply(lambda x: all(gene in genes for gene in x))]
print(combinations.shape[0])

C:\Users\Sigve\miniconda3\envs\Master\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


537499
161855


In [12]:
# Finds essential genes and gets the IDs
essential_genes = cobra.flux_analysis.find_essential_genes(model, processes=8)
essential_genes = [gene.id for gene in essential_genes]

In [5]:
# Removes all combinations containing essential genes
combinations = combinations[combinations['gene_ids'].apply(lambda x: all(gene not in essential_genes for gene in x))]
print('Number of essential genes: ' + str(len(essential_genes)))
print(combinations.shape[0])
print(combinations.head())

Number of essential genes: 137
123523
       phewas_code                                           gene_ids  \
490614      290.11  [ENSG00000033867, ENSG00000177000, ENSG0000008...   
493511      290.11  [ENSG00000099810, ENSG00000033867, ENSG0000008...   
493512      290.11  [ENSG00000099810, ENSG00000033867, ENSG0000017...   
493513      290.11  [ENSG00000099810, ENSG00000033867, ENSG0000017...   
493514      290.11  [ENSG00000099810, ENSG00000033867, ENSG0000017...   

                                                  snp_ids  
490614       ['rs4973768;rs1801133;rs11568563;rs2234978']  
493511       ['rs7023329;rs4973768;rs11568563;rs2234978']  
493512        ['rs7023329;rs4973768;rs1801133;rs2234978']  
493513       ['rs7023329;rs4973768;rs1801133;rs11568563']  
493514  ['rs7023329;rs4973768;rs1801133;rs11568563;rs2...  


In [77]:
# FBA code cell

combinations_list = []
n=3000
for g, df in combinations.groupby(np.arange(len(combinations)) // n):
    combinations_list.append(df)
print(combinations_list[0].shape[0])
print(len(combinations_list))

44
1


In [7]:
start_time = time.time()
num = 1
for sub_df in combinations_list:
    temp_df = parallelize_dataframe(sub_df, partial(combinations_subset, partial(knockout_FBA, model)), 12)
    temp_df.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/results_FBA/result_{0}.tsv'.format(num), sep='\t')
    num += 1

MaybeEncodingError: Error sending result: '[       phewas_code  ...                             results
737451      290.11  ...  <Solution 87.693 at 0x137d1e44d00>
737452      290.11  ...  <Solution 87.693 at 0x137c6d5a8e0>
737453      290.11  ...  <Solution 87.693 at 0x137c6d5ae50>
737460      290.11  ...  <Solution 87.693 at 0x137c6d5ac70>
737461      290.11  ...  <Solution 87.693 at 0x137c3439c40>
...            ...  ...                                 ...
739649      290.11  ...  <Solution 87.693 at 0x137c08c6dc0>
739650      290.11  ...  <Solution 87.693 at 0x137c08b2130>
739651      290.11  ...  <Solution 87.693 at 0x137c08b2a00>
739652      290.11  ...  <Solution 87.693 at 0x137c08b2b20>
739653      290.11  ...  <Solution 87.693 at 0x137c08b27f0>

[834 rows x 4 columns]]'. Reason: 'OSError(22, 'Insufficient system resources exist to complete the requested service', None, 1450, None)'

In [27]:
# Old
# Does not work if model imported as sbml for some reason, sometimes does though, apparently

combinations = parallelize_dataframe(combinations, partial(combinations_subset, partial(knockout_FBA, model)), 8)

# Code for single thread FBA
#combinations['results'] = combinations['gene_ids'].apply(partial(knockout_FBA, model))


combinations.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/test.tsv', sep='\t')

end_time = time.time()
print('FBA run time: %.6f seconds' % (end_time - start_time))

FBA run time: 8602.690378 seconds


In [28]:
mask = combinations['results'].apply(lambda x: x.objective_value < 87)
affected_runs = combinations[mask]
affected_runs.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/affected_runs.tsv', sep='\t')

In [ ]:
# Check for non nominal results
for i, result in combinations.iterrows():
    res = result['results'].objective_value
    if int(res) < 100:
        pass

In [6]:
gene_lists = combinations['gene_ids'].to_list()
gene_lists = [ids for sublist in gene_lists for ids in sublist]

counts = pd.Series(gene_lists).value_counts()
counts

ENSG00000155657    18
ENSG00000159063    13
ENSG00000142875    13
ENSG00000185803    11
ENSG00000126883     9
                   ..
ENSG00000060762     1
ENSG00000198919     1
ENSG00000136872     1
ENSG00000075975     1
ENSG00000147548     1
Length: 537, dtype: int64

In [36]:
# Test stuff

#print(model.reactions.MAR04908.genes)
#print(model.reactions.MAR04908.gene_reaction_rule)
#print(model.genes.get_by_id('ENSG00000103024'))

#model.genes.ENSG00000103024.knock_out()
#gene_id = ['ENSG00000155657','ENSG00000159063','ENSG00000142875']

#genes = [model.genes.get_by_id(i) for i in gene_id]
gene = model.genes.get_by_id('ENSG00000176920')
gene


Gene identifier,ENSG00000176920
Name,
Memory address,0x01f202179dc0
Functional,True
In 14 reaction(s),"MAR00904, MAR08294, MAR00886, MAR00883, MAR00864, MAR00908, MAR00909, MAR08280, MAR08262, MAR08221, MAR08286, MAR00866, MAR00898, MAR08291"


In [3]:
# Read in processed SNP data
path = 'C:/Users/Sigve/Genome_Data/'
SNP_file = path + 'results/SNPs_for_FBA.tsv'
SNP_df = pd.read_table(SNP_file)

# Get gene list
genes = [model.genes.get_by_id(i) for i in set(SNP_df['gene_id'].to_list())]
print(len(genes))

single_deletion_results = cobra.flux_analysis.single_gene_deletion(model, genes, method='fba', processes=8)
#double_deletion_results = cobra.flux_analysis.double_gene_deletion(model, genes, method='fba', processes=8)

single_deletion_results.to_csv(path_or_buf=path + 'results/test_single.tsv', sep='\t')
#double_deletion_results.to_csv(path_or_buf=path + 'results/test_double.tsv', sep='\t')

1311


In [13]:
# Remove single genes deletions that break the model before doing double deletion.
print(single_deletion_results.shape[0])
single_deletion_results = single_deletion_results[single_deletion_results['growth'] != 0]
genes = single_deletion_results['ids'].to_list()
print(genes)
genes2 = [item for sublist in genes for item in sublist]
print(len(genes2))
print(genes2)

1261
[{'ENSG00000052802'}, {'ENSG00000124006'}, {'ENSG00000197249'}, {'ENSG00000092068'}, {'ENSG00000146648'}, {'ENSG00000114738'}, {'ENSG00000165282'}, {'ENSG00000174886'}, {'ENSG00000169375'}, {'ENSG00000156515'}, {'ENSG00000079215'}, {'ENSG00000160752'}, {'ENSG00000164116'}, {'ENSG00000111817'}, {'ENSG00000152952'}, {'ENSG00000173540'}, {'ENSG00000184470'}, {'ENSG00000166484'}, {'ENSG00000149925'}, {'ENSG00000010404'}, {'ENSG00000152642'}, {'ENSG00000168487'}, {'ENSG00000112541'}, {'ENSG00000100596'}, {'ENSG00000177239'}, {'ENSG00000165349'}, {'ENSG00000196090'}, {'ENSG00000138061'}, {'ENSG00000017260'}, {'ENSG00000143199'}, {'ENSG00000108474'}, {'ENSG00000137700'}, {'ENSG00000145284'}, {'ENSG00000126522'}, {'ENSG00000060069'}, {'ENSG00000043514'}, {'ENSG00000171100'}, {'ENSG00000198886'}, {'ENSG00000139428'}, {'ENSG00000137868'}, {'ENSG00000198727'}, {'ENSG00000116096'}, {'ENSG00000171503'}, {'ENSG00000133216'}, {'ENSG00000069431'}, {'ENSG00000068438'}, {'ENSG00000160868'}, {'ENSG0

In [11]:
SNPs = pd.read_table('C:/Users/Sigve/Genome_Data/SNP_data/ALS_variants/ALS_15_loci.tsv')
SNPs.drop(columns='var_src', inplace=True)
SNPs['ancestral'] = SNPs['var'].apply(lambda x: x[0])
SNPs['var'] = SNPs['var'].apply(lambda x: x.split('/')[1:])
SNPs = SNPs.explode('var', ignore_index=True)
SNPs['var'] = SNPs['ancestral'] + '/' + SNPs['var']
SNPs.drop(columns='ancestral', inplace=True)
print(SNPs.head())

SNPs.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/SNP_data/ALS_variants/ALS_15_loci_edit.tsv', sep='\t')

       snp_id  chr        pos  var
0  rs10280711    7  157689088  G/A
1  rs10280711    7  157689088  G/C
2  rs10280711    7  157689088  G/T
3  rs10463311    5  151031274  C/A
4  rs10463311    5  151031274  C/G


In [25]:
with model:
    #model.genes.get_by_id('ENSG00000160226').knock_out()
    #model.genes.get_by_id('ENSG00000142168').knock_out()
    print(model.optimize(objective_sense='maximize'))

KeyError: 'ENSG00000142168'

In [21]:
phewas = pd.read_csv('C:/Users/Sigve/Genome_Data/SNP_data/phewas/phewas-catalog.csv')

print(phewas.head())

phewas_ids = phewas['snp']
phewas_ids = phewas_ids.unique()
phewas_ids = pd.Series(phewas_ids)

print(type(phewas_ids))
print(phewas_ids[0:10])
print(len(phewas_ids))
phewas_ids.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/SNP_data/phewas/phewas_ids.tsv', sep='\t', index=False)

    chromosome         snp                          phewas phenotype  cases  \
0  19 45395619   rs2075650                       Alzheimer's disease    737   
1  19 45395619   rs2075650                                 Dementias   1170   
2     6 396321  rs12203592                         Actinic keratosis   2505   
3   6 26093141   rs1800562                  Iron metabolism disorder     40   
4  19 45395619   rs2075650  Delirium dementia and amnestic disorders   1566   

        p-value  odds-ratio gene_name  phewas code  \
0  5.237000e-28       2.410    TOMM40       290.11   
1  2.409000e-26       2.114    TOMM40       290.10   
2  4.141000e-26       1.691      IRF4       702.10   
3  3.409000e-25      12.270       HFE       275.10   
4  8.027000e-24       1.841    TOMM40       290.00   

                                   gwas-associations  
0  Alzheimer's disease, Alzheimer's disease bioma...  
1  Alzheimer's disease, Alzheimer's disease bioma...  
2  Eye color, Hair color, Freckling

In [5]:
phewas = pd.read_csv('C:/Users/Sigve/Genome_Data/SNP_data/phewas/phewas-catalog.csv')
phewas_var = pd.read_table('C:/Users/Sigve/Genome_Data/SNP_data/phewas/phewas_variants.tsv')

phewas = phewas.merge(phewas_var, left_on='snp', right_on='Variant name')
#phewas = phewas[['snp', 'Chromosome/scaffold name','Chromosome/scaffold position start (bp)', 'Variant alleles', 'phewas code']]
phewas.rename(columns={'snp': 'snp_id','Chromosome/scaffold name': 'chr', 'Variant alleles': 'var', 'Chromosome/scaffold position start (bp)': 'pos', 'phewas code': 'phewas_code'}, inplace=True)

phewas['phewas_code'] = phewas['phewas_code'].apply(lambda x: str(x))
#phewas = phewas.groupby(['snp', 'chr', 'pos', 'var'])['phewas_code'].apply(', '.join).reset_index()

reference_phewas = phewas.drop(columns=['Variant name', 'chromosome', 'cases', 'gwas-associations'])
phewas = reference_phewas[['snp_id', 'chr', 'pos', 'var']].drop_duplicates(subset=['snp_id']).sort_values(by=['chr', 'pos'])

phewas['ancestral'] = phewas['var'].apply(lambda x: x[0])
phewas['var'] = phewas['var'].apply(lambda x: x.split('/')[1:])
phewas = phewas.explode('var', ignore_index=True)
phewas['var'] = phewas['ancestral'] + '/' + phewas['var']
phewas.drop(columns='ancestral', inplace=True)

phewas.reset_index(inplace=True, drop=True)

print(phewas.shape[0])

#reference_phewas.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/SNP_data/phewas/phewas_ref.tsv', sep='\t', index=False)
#phewas.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/SNP_data/phewas/phewas_extracted_all.tsv', sep='\t', index=False)
#temp = reference_phewas[reference_phewas['phewas_code']==290.11]
#temp.sort_values(by=['chr', 'pos'], inplace=True)
#temp.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/SNP_data/phewas/test.tsv', sep='\t', index=False)

res = phewas[phewas['snp_id'].isin(reference_phewas[reference_phewas['phewas_code'].str.contains('250.\d*')]['snp_id'])]

res.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/SNP_data/phewas/phewas_extracted_250.tsv', sep='\t', index=False)
print(res.head())

5002
      snp_id chr      pos  var
1  rs3753242   1  2138242  C/T
4  rs3748816   1  2595307  A/G
5  rs3748816   1  2595307  A/T
7  rs2651899   1  3167148  T/A
8  rs2651899   1  3167148  T/C


In [10]:
# Filter genes by submodel
genes = model.genes
genes = [gene.id for gene in genes]
print(genes)
filtered_SNPs = pd.read_table('C:/Users/Sigve/Genome_Data/results/SNPs_for_FBA.tsv', index_col=0)

filtered_SNPs = filtered_SNPs[filtered_SNPs['gene_id'].str.contains('|'.join(genes), regex=True)]
filtered_SNPs.reset_index(inplace=True, drop=True)
filtered_SNPs.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/SNPs_for_FBA.tsv',  sep='\t')

['ENSG00000000419', 'ENSG00000000938', 'ENSG00000001036', 'ENSG00000001084', 'ENSG00000001630', 'ENSG00000002549', 'ENSG00000002587', 'ENSG00000003137', 'ENSG00000003987', 'ENSG00000003989', 'ENSG00000004455', 'ENSG00000004468', 'ENSG00000004660', 'ENSG00000004779', 'ENSG00000004799', 'ENSG00000004864', 'ENSG00000005022', 'ENSG00000005075', 'ENSG00000005249', 'ENSG00000005381', 'ENSG00000005469', 'ENSG00000005483', 'ENSG00000005882', 'ENSG00000006062', 'ENSG00000006071', 'ENSG00000006432', 'ENSG00000006530', 'ENSG00000006534', 'ENSG00000006625', 'ENSG00000006756', 'ENSG00000006757', 'ENSG00000006837', 'ENSG00000007001', 'ENSG00000007047', 'ENSG00000007168', 'ENSG00000007171', 'ENSG00000007264', 'ENSG00000007541', 'ENSG00000008118', 'ENSG00000008128', 'ENSG00000008130', 'ENSG00000008300', 'ENSG00000008311', 'ENSG00000008394', 'ENSG00000008513', 'ENSG00000009413', 'ENSG00000010219', 'ENSG00000010256', 'ENSG00000010379', 'ENSG00000010404', 'ENSG00000010810', 'ENSG00000011009', 'ENSG000000

In [27]:
import re
#item = model.reactions.get_by_id('MAR03905')
#item.reaction
#print(model.optimize())
mets = model.metabolites
p=re.compile('^\w+s\[s]')

mets = [met for met in mets if p.match(met.id)]
print(mets)


[<Metabolite m00003s[s] at 0x1f03c380dc0>, <Metabolite m00008s[s] at 0x1f03c380f40>, <Metabolite m00010s[s] at 0x1f03c39c250>, <Metabolite m00017s[s] at 0x1f03c39c3a0>, <Metabolite m00019s[s] at 0x1f03c39c160>, <Metabolite m00021s[s] at 0x1f03c39c790>, <Metabolite m00094s[s] at 0x1f07f6ea490>, <Metabolite m00097s[s] at 0x1f07f6ead60>, <Metabolite m00104s[s] at 0x1f07f6fa040>, <Metabolite m00111s[s] at 0x1f07f6fa520>, <Metabolite m00114s[s] at 0x1f07f6fa5e0>, <Metabolite m00115s[s] at 0x1f07f6fa730>, <Metabolite m00117s[s] at 0x1f07f6fa820>, <Metabolite m00128s[s] at 0x1f07f6fae80>, <Metabolite m00132s[s] at 0x1f07f6fae20>, <Metabolite m00135s[s] at 0x1f07f70d0a0>, <Metabolite m00157s[s] at 0x1f07f70d3a0>, <Metabolite m00204s[s] at 0x1f07f71c220>, <Metabolite m00228s[s] at 0x1f07f71c490>, <Metabolite m00234s[s] at 0x1f07f71c4f0>, <Metabolite m00241s[s] at 0x1f07f71c6a0>, <Metabolite m00242s[s] at 0x1f07f71c700>, <Metabolite m00260s[s] at 0x1f07f71cca0>, <Metabolite m00265s[s] at 0x1f07f

In [37]:
rxns = model.reactions.get_by_id('EX_2hyoxplac[e]')
print(rxns.metabolites)
print(rxns)
print(rxns.bounds)

{<Metabolite m00654s[s] at 0x1f07fb4d7c0>: -1.0}
EX_2hyoxplac[e]: m00654s[s] <=> 
(-1000.0, 1000.0)


In [42]:
boundary_reactions = [rx for rx in model.reactions if rx.boundary]
print(len(boundary_reactions))
counter = 0

p=re.compile('^\w+s\[s]')
boundary_reactions_2 = []
for rx in model.reactions:

    if any([met for met in rx.metabolites if p.match(met.id)]):
        boundary_reactions_2.append(rx)
        counter +=1

print(counter)

print(boundary_reactions_2[1:10])
print(model.reactions.HMR_4837)

866
2622
[<Reaction HMR_4837 at 0x1f0229dffd0>, <Reaction HMR_8569 at 0x1f0229f78b0>, <Reaction HMR_8571 at 0x1f0229f7c40>, <Reaction HMR_8573 at 0x1f0229feaf0>, <Reaction HMR_8575 at 0x1f0229fe7f0>, <Reaction HMR_8577 at 0x1f0229f0b50>, <Reaction HMR_8579 at 0x1f022a08ac0>, <Reaction HMR_8581 at 0x1f022a08d90>, <Reaction HMR_4303 at 0x1f022a15ee0>]
HMR_4837: m02040s[s] + m03039s[s] --> 2.0 m01965s[s]


In [51]:
rxns = []
for rx in model.reactions:
    mets = list(rx.metabolites.keys())
    if len(mets) == 2:
        if mets[0].id[:5] == mets[1].id[:5] and (p.match(mets[0].id) or (p.match(mets[1].id))):
            print(mets)
            rxns.append(rx)
print(len(rxns))

[<Metabolite m02035c[c] at 0x1f07f8e9f10>, <Metabolite m02035s[s] at 0x1f07f8e9f40>]
[<Metabolite m00503s[s] at 0x1f07f753eb0>, <Metabolite m00503c[c] at 0x1f07f753ee0>]
[<Metabolite m00510s[s] at 0x1f07f763220>, <Metabolite m00510c[c] at 0x1f07f7631f0>]
[<Metabolite m01451s[s] at 0x1f07f843b20>, <Metabolite m01451l[l] at 0x1f07f843af0>]
[<Metabolite m01410s[s] at 0x1f07f8310a0>, <Metabolite m01410c[c] at 0x1f07f831d00>]
[<Metabolite m03134s[s] at 0x1f07fa11160>, <Metabolite m03134c[c] at 0x1f07fa11130>]
[<Metabolite m02120s[s] at 0x1f07f90aca0>, <Metabolite m02120c[c] at 0x1f07f90ac70>]
[<Metabolite m02108s[s] at 0x1f07f90a7c0>, <Metabolite m02108c[c] at 0x1f07f90a790>]
[<Metabolite m02642s[s] at 0x1f07f98f5b0>, <Metabolite m02642c[c] at 0x1f07f98f580>]
[<Metabolite m02614s[s] at 0x1f07f97ec40>, <Metabolite m02614c[c] at 0x1f07f97ec10>]
[<Metabolite m01648s[s] at 0x1f07f885b50>, <Metabolite m01648c[c] at 0x1f07f885b20>]
[<Metabolite m03117s[s] at 0x1f07fa00d30>, <Metabolite m03117c[c]

In [19]:
print(len(model.boundary))
print(len(model.exchanges))
print(model.reactions.get_by_id('MAR13086'))

866
866


KeyError: 'MAR13086'

In [ ]:
L = ['HMR_10024', 'HMR_10026', 'HMR_10027', 'HMR_10047']
for i in L:
    print(model.reactions.get_by_id(i))
    print([met.name for met in list(model.reactions.get_by_id(i).metabolites.keys())])
    print('------------------------------')

In [53]:
# Validation, takes a long time.
from pprint import pprint
from cobra.io import write_sbml_model, validate_sbml_model

report = validate_sbml_model('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/brain.xml')

pprint(report)

(<Model M_brain at 0x1f02c8eebb0>,
 {'COBRA_CHECK': [],
  'COBRA_ERROR': [],
  'COBRA_FATAL': [],
  'COBRA_WARNING': [],
  'SBML_ERROR': [],
  'SBML_FATAL': [],
  'SBML_SCHEMA_ERROR': [],
  'SBML_WARNING': []})


In [2]:
# Make a custom ID and print to file.
snp_data = pd.read_table('C:/Users/Sigve/Genome_Data/SNP_data/1000_genomes/result_chrom_22.tsv', index_col=0)

In [11]:
snp_data['snp_id'] = snp_data['pos'].apply(lambda x: '22:' + str(x))
snp_data['chr'] = '22'

snp_data[['snp_id', 'chr', 'pos', 'var']].set_index('snp_id').to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/SNP_data/1000_genomes/result_chrom_22_snps.tsv', sep='\t')


In [12]:
# Try to get Entrez ids below, it is shit
from Bio import Entrez
missense_snps = pd.read_table('C:/Users/Sigve/Genome_Data/results/SNPs_missense.tsv', index_col=0)

In [18]:
def entrez_get(pos):
    Entrez.email = 'sigve.landa@outlook.com'
    handle = Entrez.esearch(db='snp', term='22[Chromosome] AND ' + str(pos) + '[CHRPOS]')
    records = Entrez.read(handle)
    handle.close()
    return(records['IdList'])


missense_snps['variant_name'] = missense_snps['chrom_pos'].apply(lambda x: entrez_get(x))
print(missense_snps.head())

                   variant_name  chrom  chrom_pos variant_alleles gene_name  \
0                rs['45497794']     22   17181554             C/G      ADA2   
1                rs['61744537']     22   17181877             A/G      ADA2   
2  rs['758232786', '186147069']     22   17181959             G/C      ADA2   
3               rs['561591791']     22   17182708             T/C      ADA2   
4                rs['45511697']     22   17188348             C/T      ADA2   

              gene_id      transcript_id          exon_id amino_acid_change  \
0  ENSG00000093072.18  ENST00000399837.8  ENSE00001540382               E/Q   
1  ENSG00000093072.18  ENST00000399837.8  ENSE00000650248               I/T   
2  ENSG00000093072.18  ENST00000399837.8  ENSE00000650248               P/A   
3  ENSG00000093072.18  ENST00000399837.8  ENSE00000650250               T/A   
4  ENSG00000093072.18  ENST00000399837.8  ENSE00000650252               G/R   

   amino_acid_pos  protein_length  score  
0      

In [35]:
missense_snps['variant_name'] = missense_snps['variant_name'].apply(lambda x: x[4:-2].split("', '"))

AttributeError: 'list' object has no attribute 'split'

In [36]:
test = missense_snps[missense_snps['variant_name'].apply(len) > 1]
print(test)

                                           variant_name  chrom  chrom_pos  \
2                                [758232786, 186147069]     22   17181959   
5                                [117224542, 116020027]     22   17188355   
8                        [117733135, 61542193, 2231495]     22   17188416   
63                                   [3210451, 3180408]     22   18167915   
90                         [17743850, 3171566, 1052763]     22   19132238   
...                                                 ...    ...        ...   
1972                             [761555167, 117341984]     22   50625330   
1974                               [386601777, 6151427]     22   50625350   
1979  [80338810, 60738949, 17856452, 17184315, 31717...     22   50625611   
1983                      [80338809, 61664838, 2071421]     22   50625988   
1990                               [80338818, 74315471]     22   50626706   

     variant_alleles gene_name             gene_id       transcript_id  \
2

In [47]:
# Produce combinations based on individual data
ind_data = pd.read_table('C:/Users/Sigve/Genome_Data/SNP_data/1000_genomes/result_chrom_22.tsv', index_col=0)
missense_snps = pd.read_table('C:/Users/Sigve/Genome_Data/results/SNPs_missense.tsv', index_col=0)

In [62]:
#(Also filter by model genes and essential genes here?)
missense_snps['gene_id'] = missense_snps['gene_id'].apply(lambda x: x.split('.')[0])
merged_df = pd.merge(left=missense_snps, right=ind_data, how='left', left_on='chrom_pos', right_on='pos')


HG00096
  variant_name  chrom  chrom_pos variant_alleles gene_name          gene_id  \
0  22:17181554     22   17181554             C/G      ADA2  ENSG00000093072   
1  22:17181877     22   17181877             A/G      ADA2  ENSG00000093072   
2  22:17181959     22   17181959             G/C      ADA2  ENSG00000093072   
3  22:17182708     22   17182708             T/C      ADA2  ENSG00000093072   
4  22:17188348     22   17188348             C/T      ADA2  ENSG00000093072   

       transcript_id          exon_id amino_acid_change  amino_acid_pos  ...  \
0  ENST00000399837.8  ENSE00001540382               E/Q           489.0  ...   
1  ENST00000399837.8  ENSE00000650248               I/T           462.0  ...   
2  ENST00000399837.8  ENSE00000650248               P/A           435.0  ...   
3  ENST00000399837.8  ENSE00000650250               T/A           379.0  ...   
4  ENST00000399837.8  ENSE00000650252               G/R           358.0  ...   

   NA21128  NA21129  NA21130 NA21133

In [97]:
res_dict = {}
n=0
for column in merged_df:
    n +=1
    if n < 15:
        continue

    res_dict[column] = [merged_df['variant_name'][merged_df[column] == 2].to_list(), set(merged_df['gene_id'][merged_df[column] == 2].to_list())]

results = pd.DataFrame.from_dict(res_dict, orient='index', columns=['snp_list', 'gene_ids'])
print(results.head())

                                                  snp_list  \
HG00096  [22:19880695, 22:20364235, 22:20431352, 22:232...   
HG00097  [22:19880695, 22:30557308, 22:31140147, 22:380...   
HG00099  [22:18167915, 22:19132238, 22:19919576, 22:203...   
HG00100  [22:19880695, 22:20425478, 22:31140147, 22:499...   
HG00101  [22:19880695, 22:20364235, 22:23285182, 22:378...   

                                                  gene_ids  
HG00096  {ENSG00000128274, ENSG00000205560, ENSG0000016...  
HG00097  {ENSG00000205560, ENSG00000184470, ENSG0000010...  
HG00099  {ENSG00000206203, ENSG00000205560, ENSG0000016...  
HG00100  {ENSG00000100078, ENSG00000184470, ENSG0000019...  
HG00101  {ENSG00000184983, ENSG00000161133, ENSG0000018...  


In [98]:
#genes
#essential_genes



results['gene_ids'] = results['gene_ids'].apply(lambda x: list(x.intersection(genes)))
results = results[results['gene_ids'].map(lambda x: len(x)) > 0]
results = results[results['gene_ids'].apply(lambda x: all(gene not in essential_genes for gene in x))]

results.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/ind_combinations.tsv', sep='\t')





In [4]:
genes = model.genes
genes = [gene.id for gene in genes]

print(len(genes))
genes = [gene for gene in genes if gene not in essential_genes]
print(len(genes))
length = len(genes)

2451
2314


In [27]:
# Combination generator
import random
result = {}

SNPs = pd.Series(genes)

for i in range(200):
    sample = random.sample(range(0, length), 20)

    # Gets the SNP IDs to a list.
    sample_SNPs = SNPs.filter(items=sample, axis=0)

    # Gets the ensemble IDs to a list, and removes the version number.
    sample_gene_ids = list(set(sample_SNPs.apply(lambda x: x.split('.')[0]).tolist()))
    result[i] = [sample_gene_ids]

pd.DataFrame.from_dict(result, orient='index', columns=["gene_ids"]).to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/SNP_combinations_model.tsv', sep='\t')



In [25]:
combinations = pd.DataFrame.from_dict(result, orient='index', columns=["gene_ids"])
combinations = parallelize_dataframe(combinations, partial(combinations_subset, partial(knockout_FBA, model)), 8)

In [26]:
combinations.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/model_test.tsv', sep='\t')

In [74]:
combinations = pd.read_table('C:/Users/Sigve/Genome_Data/results/model_tests/model_non_ess_eff.tsv', index_col=0)
combinations['gene_ids'] = combinations['gene_ids'].apply(lambda x: x[2:-2])
print(combinations.head())
S = list(combinations['gene_ids'])
print(len(S))

          gene_ids                             results
0  ENSG00000004468  <Solution 86.250 at 0x1c6117e50d0>
1  ENSG00000006530  <Solution 87.687 at 0x1c60c1adb50>
2  ENSG00000012779  <Solution 87.380 at 0x1c5e96e7a60>
3  ENSG00000023697  <Solution 81.150 at 0x1c61629c4f0>
4  ENSG00000065154  <Solution 87.464 at 0x1c616ebeb80>
44


In [75]:
res = []
for i in range(3, 4):
    for j in itertools.combinations(S, i):
        res.append([list(j)])
print(len(res))


13244


In [76]:
pd.DataFrame(res, columns=['gene_ids']).to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/model_tests/model_non_ess_eff_comb_3.tsv', sep='\t')

In [70]:
res = [[sub.tolist()] for sub in np.array_split(res, 3)]
print(len(res))
print(res)

3
[[['ENSG00000004468', 'ENSG00000006530', 'ENSG00000012779'], ['ENSG00000004468', 'ENSG00000006530', 'ENSG00000023697'], ['ENSG00000004468', 'ENSG00000006530', 'ENSG00000065154'], ['ENSG00000004468', 'ENSG00000006530', 'ENSG00000075415'], ['ENSG00000004468', 'ENSG00000006530', 'ENSG00000102743'], ['ENSG00000004468', 'ENSG00000006530', 'ENSG00000103222'], ['ENSG00000004468', 'ENSG00000006530', 'ENSG00000105220'], ['ENSG00000004468', 'ENSG00000006530', 'ENSG00000108528'], ['ENSG00000004468', 'ENSG00000006530', 'ENSG00000111640'], ['ENSG00000004468', 'ENSG00000006530', 'ENSG00000111669'], ['ENSG00000004468', 'ENSG00000006530', 'ENSG00000125246'], ['ENSG00000004468', 'ENSG00000006530', 'ENSG00000125772'], ['ENSG00000004468', 'ENSG00000006530', 'ENSG00000125877'], ['ENSG00000004468', 'ENSG00000006530', 'ENSG00000130005'], ['ENSG00000004468', 'ENSG00000006530', 'ENSG00000130649'], ['ENSG00000004468', 'ENSG00000006530', 'ENSG00000130821'], ['ENSG00000004468', 'ENSG00000006530', 'ENSG00000133

In [60]:
combinations = pd.DataFrame(res, columns=['gene_ids'])
combinations = parallelize_dataframe(combinations, partial(combinations_subset, partial(knockout_FBA, model)), 6)

In [61]:
combinations.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/model_tests/model_test_1.tsv', sep='\t')

In [77]:
for i, result in combinations.iterrows():
    res = result['results'].objective_value
    if int(res) < 80:
        print(result)

In [15]:
gene_list = [[[gene]] for gene in genes if gene not in essential_genes]
pd.DataFrame(gene_list, columns=['gene_ids']).to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/model_tests/model_non_ess_genes.tsv', sep='\t')

In [31]:
essential = pd.read_csv('C:/Users/Sigve/Genome_Data/results/model_tests/brain_model_essential.csv', index_col=0)

essential['pass/fail'] = essential.values.tolist()
essential = essential[['pass/fail']].reset_index()
print(essential.shape[0])
essential = essential[essential['Var1'].map(lambda x: x not in essential_genes)]
print(essential.shape[0])
essential = essential[essential['pass/fail'].map(lambda x: not any(task == 1 for task in x))]
essential['gene_ids'] = essential['Var1'].apply(lambda x: [x])
essential = pd.DataFrame(essential['gene_ids'])
essential.reset_index(inplace=True, drop=True)
print(essential.head())
print(essential.shape[0])


essential.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/model_tests/model_non_ess_genes_ext.tsv', sep='\t')

2451
2314
            gene_ids
0  [ENSG00000000419]
1  [ENSG00000000938]
2  [ENSG00000001036]
3  [ENSG00000002549]
4  [ENSG00000002587]
2163


In [36]:
result = parallelize_dataframe(essential, partial(combinations_subset, partial(knockout_FBA, model)), 8)

In [40]:
for i, resu in result.iterrows():
    res = resu['results'].objective_value
    if float(res) < 87.693:
        print(resu)

gene_ids                     [ENSG00000004468]
results     <Solution 86.250 at 0x1c6117e50d0>
Name: 8, dtype: object
gene_ids                     [ENSG00000006530]
results     <Solution 87.687 at 0x1c60c1adb50>
Name: 21, dtype: object
gene_ids                     [ENSG00000012779]
results     <Solution 87.380 at 0x1c5e96e7a60>
Name: 54, dtype: object
gene_ids                     [ENSG00000023697]
results     <Solution 81.150 at 0x1c61629c4f0>
Name: 74, dtype: object
gene_ids                     [ENSG00000065154]
results     <Solution 87.464 at 0x1c616ebeb80>
Name: 152, dtype: object
gene_ids                     [ENSG00000075415]
results     <Solution 82.193 at 0x1c5e71da040>
Name: 239, dtype: object
gene_ids                     [ENSG00000102743]
results     <Solution 87.464 at 0x1c5e7db62e0>
Name: 436, dtype: object
gene_ids                     [ENSG00000103222]
results     <Solution 87.380 at 0x1c605bf1c40>
Name: 447, dtype: object
gene_ids                     [ENSG00000105220]
result

In [45]:
aff = result[result['results'].map(lambda x: float(x.objective_value) < 87.693)].reset_index(drop=True)
print(aff)
print(aff.shape[0])

aff.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/model_tests/model_non_ess_eff.tsv', sep='\t')

             gene_ids                             results
0   [ENSG00000004468]  <Solution 86.250 at 0x1c6117e50d0>
1   [ENSG00000006530]  <Solution 87.687 at 0x1c60c1adb50>
2   [ENSG00000012779]  <Solution 87.380 at 0x1c5e96e7a60>
3   [ENSG00000023697]  <Solution 81.150 at 0x1c61629c4f0>
4   [ENSG00000065154]  <Solution 87.464 at 0x1c616ebeb80>
5   [ENSG00000075415]  <Solution 82.193 at 0x1c5e71da040>
6   [ENSG00000102743]  <Solution 87.464 at 0x1c5e7db62e0>
7   [ENSG00000103222]  <Solution 87.380 at 0x1c605bf1c40>
8   [ENSG00000105220]  <Solution 86.695 at 0x1c604b49c40>
9   [ENSG00000108528]  <Solution 86.882 at 0x1c601e82d30>
10  [ENSG00000111640]  <Solution 70.988 at 0x1c610bffc40>
11  [ENSG00000111669]  <Solution 84.282 at 0x1c610bf6eb0>
12  [ENSG00000125246]  <Solution 85.965 at 0x1c610e20af0>
13  [ENSG00000125772]  <Solution 87.692 at 0x1c5e84aad90>
14  [ENSG00000125877]  <Solution 82.526 at 0x1c60efde040>
15  [ENSG00000130005]  <Solution 87.679 at 0x1c604d85730>
16  [ENSG00000

In [78]:
combinations = pd.DataFrame(res, columns=['gene_ids'])
combinations_list = []
n=3000
for g, df in combinations.groupby(np.arange(len(combinations)) // n):
    combinations_list.append(df)
print(combinations_list[0].shape[0])
print(len(combinations_list))

3000
5


In [79]:
start_time = time.time()
num = 1
for sub_df in combinations_list:
    temp_df = parallelize_dataframe(sub_df, partial(combinations_subset, partial(knockout_FBA, model)), 6)
    temp_df.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/results_FBA/result_{0}.tsv'.format(num), sep='\t')
    num += 1

print('FBA run time: %.6f seconds' % (end_time - start_time))


FBA run time: -20642.769544 seconds
